In [2]:
## All baselines
import os
#from CKA import linear_CKA, kernel_CKA
#import cca_core
import json
import numpy as np
from sklearn.model_selection import train_test_split,KFold
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pickle 
from tensorflow.keras.datasets import mnist, fashion_mnist
import tensorflow as tf
import random
from tensorflow.keras import backend as K
#from tensorflow import set_random_seed
from tensorflow.keras.losses import MeanSquaredError

#Importing the necessary packages and libaries
from sklearn.metrics import confusion_matrix
from sklearn import svm, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [1]:
!pip install tf-nightly

In [ ]:
# Random seed function
def reset_random_seeds():
    seed_value=1
    os.environ['PYTHONHASHSEED']=str(seed_value)
    tf.random.set_random_seed(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    set_random_seed(seed_value)
    session_conf=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess=tf.Session(graph=tf.get_default_graph(),config=session_conf)
    K.set_session(sess)

# load data
def load_data(dataset_path):
    with open(dataset_path,"r") as fp:
        data=json.load(fp)  #load the dictionary from the file
        
    # convert list into numpy array
    inputs=np.array(data["mfcc"])
    targets=np.array(data["labels"]) #.astype(int)
    #targets=gender_to_digit(targets)
    return inputs,targets

def prepare_datasets(test_size,DATASET_PATH,ln):
    #load data
    X,y=load_data(DATASET_PATH)
    if ln=="sw":
        X,y=sort_dataset(X,y)
    
    #create train/test split
    #split the data into train and test set
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=test_size)
    X_train=X_train[...,np.newaxis] #4d array-> (num_samples,130,13,1)
    X_test=X_test[...,np.newaxis]
        
    return X_train,y_train,X_test,y_test
# resize Swahili and English datasets size(3000 utterances) to Gujarati size (1940 utterances)
def resize_to_gu_size(X,y):
    y_l=[]
    X_l=[]
    for i in range(0,2702,300):
        if i!=0:
            y_l.append(y[i+1:i+1+194])
            X_l.append(X[i+1:i+1+194])
        else:
            y_l.append(y[i:194])
            X_l.append(X[i:194])
    
    X_l=np.array(X_l)
    y_l=np.array(y_l)
    X_l=X_l.reshape(1940,64,16)
    y_l=y_l.reshape(1940,)
    return X_l,y_l

# Sort datasets y[0,2,9,0,0,3,....,4,8,1] -> y[0,0,0,1,1,1,2,2,2.....9,9,9] and its corresponding Xs[....] 
def sort_dataset(X,y):    
    #convert into list from numpy
    X=list(X)
    y=list(y)
    #zip
    data1 = list(zip(y, X))
    #sort multiple list in the zip
    data1=sorted(data1, key=lambda x: x[0])
    #unzip
    y_s, X_s = zip(*data1)
    #convert back to numpy from tuple/list
    y_s=np.asarray(y_s)
    X_s=np.asarray(X_s)
    
    return X_s,y_s

# Section 5.1 
# ln -language
def prepare_datasets_gu(test_size,DATASET_PATH,ln):
    # load data
    X,y=load_data(DATASET_PATH)
    # re-sort dataset
    X,y=sort_dataset(X,y)
    if ln=="swen": #select language
        X,y=resize_to_gu_size(X,y)  #resize to 1940 utterances
    
    #create train/test split
    #split the data into train and test set
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=test_size)
    X_train=X_train[...,np.newaxis] #4d array-> (num_samples,130,13,1)
    X_test=X_test[...,np.newaxis]
        
    return X_train,y_train,X_test,y_test

def plot_history(history):
    fig,axs=plt.subplots(2)
    
    #create accuracy suplot
    axs[0].plot(history.history["acc"],label="train accuracy")
    axs[0].plot(history.history["val_acc"],label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")
    
    #create accuracy suplot
    axs[1].plot(history.history["loss"],label="train error")
    axs[1].plot(history.history["val_loss"],label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")
    
    plt.show()

In [ ]:
# Download the MFCCs files from https://drive.google.com/drive/folders/1x7tZIfxsTRTB-hdPx5QSLm27dq1zrx2J?usp=sharing
DATASET_PATH_SW= "MNIST-digit-sw-mfcc-norm-16.json"  #Swahili dataset file
DATASET_PATH_EN= "MNIST-digit-en-mfcc-norm-16.json"  #English dataset file
DATASET_PATH_GU= "MNIST-digit-gu-mfcc-norm-16.json"  #Gurajati dataset file

In [ ]:
# Load each dataset into its container
if __name__=="__main__":
    #inputs,targets=load_data(DATASET_PATH)
    reset_random_seeds()
        
    #Swahili Train and Test dataset for X features and y labels
    X_train_sw,y_train_sw,X_test_sw,y_test_sw=prepare_datasets_gu(0.2,DATASET_PATH_SW,"swen")
    reset_random_seeds()
    
    #English Train and Test dataset for X features and y labels
    X_train_en,y_train_en,X_test_en,y_test_en=prepare_datasets_gu(0.2,DATASET_PATH_EN,"swen")
    reset_random_seeds()
    
    #Gujarati Train and Test dataset for X features and y labels
    X_train_gu,y_train_gu,X_test_gu,y_test_gu=prepare_datasets_gu(0.2,DATASET_PATH_GU,"gu")

In [ ]:
# Reshape for svm training and evaluating
X_train_sw_svm=X_train_sw.reshape(1552, 1024)
X_test_sw_svm=X_test_sw.reshape(388,1024)

X_train_en_svm=X_train_en.reshape(1552, 1024)
X_test_en_svm=X_test_en.reshape(388,1024)

X_train_gu_svm=X_train_gu.reshape(1552, 1024)
X_test_gu_svm=X_test_gu.reshape(388,1024)

In [ ]:
X_test_sw.shape

(388, 64, 16, 1)

In [ ]:
y_train_gu

array(['3', '0', '3', ..., '9', '1', '5'], dtype='<U1')

In [ ]:
# Training and Evaluating SVM for kernels linear, RBF, 
gamma=0.5   
for i in range(20):
    
    
    # training Swahili dataset
    print("C value:",i+1)  
   # linear_sw = svm.SVC(kernel='linear', C=i+1, decision_function_shape='ovo').fit(X_train_sw_svm, y_train_sw)
    linear_sw = svm.SVC(kernel='rbf', gamma=gamma, C=i+1, decision_function_shape='ovo').fit(X_train_sw_svm, y_train_sw)
  #  linear_sw = svm.SVC(kernel='poly', degree=1, C=i+1, decision_function_shape='ovo').fit(X_train_sw_svm, y_train_sw)
    accuracy_lin_sw = linear_sw.score(X_test_sw_svm, y_test_sw)
    print("Accuracy Linear Kernel Swahili:", accuracy_lin_sw)

    # training English dataset
   # linear_en = svm.SVC(kernel='linear', C=i+1, decision_function_shape='ovo').fit(X_train_en_svm, y_train_en)
    linear_en = svm.SVC(kernel='rbf', gamma=gamma, C=i+1, decision_function_shape='ovo').fit(X_train_en_svm, y_train_en)
   # linear_en = svm.SVC(kernel='poly', degree=1, C=i+1, decision_function_shape='ovo').fit(X_train_en_svm, y_train_en)
    accuracy_lin_en = linear_en.score(X_test_en_svm, y_test_en)
    print("Accuracy Linear Kernel English:", accuracy_lin_en)

    # training Gujarati dataset
   # linear_gu = svm.SVC(kernel='linear', C=i+1, decision_function_shape='ovo').fit(X_train_gu_svm, y_train_gu)
    linear_gu = svm.SVC(kernel='rbf', gamma=gamma, C=i+1, decision_function_shape='ovo').fit(X_train_gu_svm, y_train_gu)
  #  linear_gu = svm.SVC(kernel='poly', degree=1, C=i+1, decision_function_shape='ovo').fit(X_train_gu_svm, y_train_gu)
    accuracy_lin_gu = linear_gu.score(X_test_gu_svm, y_test_gu)
    print("Accuracy Linear Kernel Gujarati:", accuracy_lin_gu)
    print ("------------------------------------------")

C value: 20
Accuracy Linear Kernel Swahili: 0.5489690721649485
Accuracy Linear Kernel English: 0.6030927835051546
Accuracy Linear Kernel Gujarati: 0.5335051546391752
------------------------------------------


In [ ]:
# Training and Evaluating Random Forest
no_tree=[10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
# Fitting Random Forest Classification to the Training set
for i in range(len(u)):
    print("no tree values:",u[i]) 
    # Training Swahili
    classifier_sw = RandomForestClassifier(n_estimators = no_tree[i], criterion = 'entropy', random_state = 42)
    classifier_sw.fit(X_train_sw_svm, y_train_sw)
    y_pred_sw = classifier_sw.predict(X_test_sw_svm)
    print("Accuracy:",metrics.accuracy_score(y_test_sw, y_pred_sw))

    # Training English
    classifier_en = RandomForestClassifier(n_estimators = no_tree[i], criterion = 'entropy', random_state = 42)
    classifier_en.fit(X_train_en_svm, y_train_en)
    y_pred_en = classifier_en.predict(X_test_en_svm)
    print("Accuracy:",metrics.accuracy_score(y_test_en, y_pred_en))

    # Training Gujarati
    classifier_gu = RandomForestClassifier(n_estimators = no_tree[i], criterion = 'entropy', random_state = 42)
    classifier_gu.fit(X_train_gu_svm, y_train_gu)
    y_pred_gu = classifier_gu.predict(X_test_gu_svm)
    print("Accuracy:",metrics.accuracy_score(y_test_gu, y_pred_gu))
    print("----------------------------------------")

no tree values: 10
Accuracy: 0.7835051546391752
Accuracy: 0.8762886597938144
Accuracy: 0.6752577319587629
----------------------------------------
no tree values: 20
Accuracy: 0.8015463917525774
Accuracy: 0.8994845360824743
Accuracy: 0.7731958762886598
----------------------------------------
no tree values: 30
Accuracy: 0.8221649484536082
Accuracy: 0.9097938144329897
Accuracy: 0.7912371134020618
----------------------------------------
no tree values: 40
Accuracy: 0.8247422680412371
Accuracy: 0.9226804123711341
Accuracy: 0.788659793814433
----------------------------------------
no tree values: 50
Accuracy: 0.8221649484536082
Accuracy: 0.9355670103092784
Accuracy: 0.788659793814433
----------------------------------------
no tree values: 60
Accuracy: 0.8402061855670103
Accuracy: 0.9381443298969072
Accuracy: 0.8015463917525774
----------------------------------------
no tree values: 70
Accuracy: 0.8479381443298969
Accuracy: 0.9407216494845361
Accuracy: 0.7912371134020618
--------------

In [ ]:
# Training and Evaluating knn

for k in range(20):
    print("k neighbors value:",k+1) 
    # Training Swahili
    knn_sw = KNeighborsClassifier(n_neighbors=k+1)
    knn_sw.fit(X_train_sw_svm, y_train_sw)
    y_pred_sw = knn_sw.predict(X_test_sw_svm)
    print("Accuracy:",metrics.accuracy_score(y_test_sw, y_pred_sw))

    # Training English
    knn_en = KNeighborsClassifier(n_neighbors=k+1)
    knn_en.fit(X_train_en_svm, y_train_en)
    y_pred_en = knn_en.predict(X_test_en_svm)
    print("Accuracy:",metrics.accuracy_score(y_test_en, y_pred_en))

    # Training Swahili
    knn_gu = KNeighborsClassifier(n_neighbors=k+1)
    knn_gu.fit(X_train_gu_svm, y_train_gu)
    y_pred_gu = knn_gu.predict(X_test_gu_svm)
    print("Accuracy:",metrics.accuracy_score(y_test_gu, y_pred_gu))
    print ("------------------------------------------")

k neighbors value: 1
Accuracy: 0.7319587628865979
Accuracy: 0.904639175257732
Accuracy: 0.7680412371134021
------------------------------------------
k neighbors value: 2
Accuracy: 0.6907216494845361
Accuracy: 0.8994845360824743
Accuracy: 0.7216494845360825
------------------------------------------
k neighbors value: 3
Accuracy: 0.6881443298969072
Accuracy: 0.8917525773195877
Accuracy: 0.7371134020618557
------------------------------------------
k neighbors value: 4
Accuracy: 0.6881443298969072
Accuracy: 0.884020618556701
Accuracy: 0.7396907216494846
------------------------------------------
k neighbors value: 5
Accuracy: 0.6829896907216495
Accuracy: 0.8634020618556701
Accuracy: 0.7242268041237113
------------------------------------------
k neighbors value: 6
Accuracy: 0.6701030927835051
Accuracy: 0.8711340206185567
Accuracy: 0.7036082474226805
------------------------------------------
k neighbors value: 7
Accuracy: 0.6752577319587629
Accuracy: 0.8685567010309279
Accuracy: 0.69845

In [ ]:
# Table 5 Multi-lingual data 
# Note: Run this script only for Multi-lingual model training

# English + Gujarati combination datasets
X_train_en_gu=np.concatenate((X_train_en,X_train_gu))
y_train_en_gu=np.concatenate((y_train_en,y_train_gu))

# Gujarati + Swahili combination datasets
X_train_gu_sw=np.concatenate((X_train_gu,X_train_sw))
y_train_gu_sw=np.concatenate((y_train_gu,y_train_sw))

# Swahili + English combination datasets
X_train_sw_en=np.concatenate((X_train_sw,X_train_en))
y_train_sw_en=np.concatenate((y_train_sw,y_train_en))

In [ ]:
len(y_train_sw_en)

3104

In [ ]:
y_train_gu

array(['3', '0', '3', ..., '9', '1', '5'], dtype='<U1')

In [ ]:
# Pre- Training model
# Parameters
epoch_l = [1,5,10,20,30]  # no of epochs
lan=['sw','en','gu']      #languages
lan_str=["Swahili","English","Gujarati"]   #languages string

# Monolingual or Crosslingual case
source=[X_train_sw,X_train_en,X_train_gu]  #source dataset features
target=[y_train_sw,y_train_en,y_train_gu]  #target dataset features

# Multi-lingual case
source_comb=[X_train_en_gu,X_train_gu_sw,X_train_sw_en]  #source dataset features
target_comb=[y_train_en_gu,y_train_gu_sw,y_train_sw_en]  #target dataset features

# Looping each source language and across no of epochs
for j in range(len(lan)):
    print("Source language: "+lan_str[j])
    for i in range(len(epoch_l)):
        reset_random_seeds()

        #change data type
        X_train_en=X_train_en.astype("float32")
        X_test_en=X_test_en.astype("float32")

        X_train_sw=X_train_sw.astype("float32")
        X_test_sw=X_test_sw.astype("float32")

        X_train_gu=X_train_gu.astype("float32")
        X_test_gu=X_test_gu.astype("float32")


        #define history container
        history=[]

        #for train, test in kfold.split(inputs, targets):
        ## Define Auto-encoder 
        latent_space_dim=64
        # Table 3
        ## Input shape definition
        input_shape=keras.layers.Input(shape=(X_train_en.shape[1], X_train_en.shape[2], X_train_en.shape[3]))

        ## Encode layers
        #1st conv layer
        conv1=keras.layers.Conv2D(32,3,(1,1),padding='same')(input_shape)
        conv1=keras.layers.ReLU()(conv1)
        conv1=keras.layers.BatchNormalization()(conv1)

        #2nd conv layer
        conv2=keras.layers.Conv2D(64,3,(2,2),padding='same')(conv1)
        conv2=keras.layers.ReLU()(conv2)
        conv2=keras.layers.BatchNormalization()(conv2)

        #3rd conv layer
        conv3=keras.layers.Conv2D(64,3,(2,2),padding='same')(conv2)
        conv3=keras.layers.ReLU()(conv3)
        conv3=keras.layers.BatchNormalization()(conv3)

        #shape before flatten
        shape_before_flatten = keras.backend.int_shape(conv3)[1:]  #conv3  ,conv2

        #flatten the output and feed it into dense layer (bottleneck)
        conv_flatten=keras.layers.Flatten()(conv3)   #conv3     ,conv2
        encoded=keras.layers.Dense(latent_space_dim,activation='relu')(conv_flatten)

        ## baseline model classifier 
        drop=keras.layers.Dropout(0.3,seed=1)(encoded)
        fc=keras.layers.Dense(10,activation='softmax')(drop)
        
        # Model
        full_model = keras.Model(input_shape,fc)

        #compile the network
        optimizer=keras.optimizers.Adam(lr=0.0001)
        full_model.compile(optimizer=optimizer,
                     loss="sparse_categorical_crossentropy",
                     metrics=['acc'])

        ## monolingual Pre-training
        filename='test-Single-L3/CNN-digit-mfcc-T3-E'+str(epoch_l[i])+'-S'+lan[j]+'.csv'
        ##track log
        history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)
        ##Training
        #full_model.save("Single-L3/CNN-digit-mfcc-T3-Single")
        print("For Epoch: "+str(epoch_l[i]))
        history=full_model.fit(source[j],target[j],batch_size=32,epochs=epoch_l[i],verbose=1,callbacks=[history_logger]) #,callbacks=[history_logger]
        
        ##Save the model
        full_model.save("test-Single-L3/CNN-digit-mfcc-T3-E"+str(epoch_l[i])+"-S"+lan[j])
        #full_model.save("Single-L3/CNN-digit-mfcc-T3-Single")
        #reset
        tf.reset_default_graph()
        
        
        
        
        
        ## multi-lingual Pre-training
#        filename='Single-L3-two/reverse/CNN-digit-mfcc-T3-E'+str(epoch_l[i])+'-S'+lan[j]+'.csv'
        ##track log
#        history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)
        ##Training
        #full_model.save("Single-L3/CNN-digit-mfcc-T3-Single")
#        print("For Epoch: "+str(epoch_l[i]))
#        history=full_model.fit(source_comb[j],target_comb[j],batch_size=32,epochs=epoch_l[i],verbose=1,callbacks=[history_logger]) #,callbacks=[history_logger]
        
        ##Save the model
#        full_model.save("Single-L3-two/reverse/CNN-digit-mfcc-T3-E"+str(epoch_l[i])+"-S"+lan[j])
        #full_model.save("Single-L3/CNN-digit-mfcc-T3-Single")
        #reset
#        tf.reset_default_graph()


Source language: Swahili
For Epoch: 1
1552/1552 [==============================] - 23s 15ms/sample - loss: 2.1634 - acc: 0.2371
For Epoch: 5
Epoch 1/5
1552/1552 [==============================] - 24s 15ms/sample - loss: 2.1634 - acc: 0.2371
Epoch 2/5
1552/1552 [==============================] - 25s 16ms/sample - loss: 1.6092 - acc: 0.4066
Epoch 3/5
1552/1552 [==============================] - 23s 15ms/sample - loss: 1.2806 - acc: 0.5329
Epoch 4/5
1552/1552 [==============================] - 23s 15ms/sample - loss: 1.0679 - acc: 0.5992
Epoch 5/5
1552/1552 [==============================] - 26s 17ms/sample - loss: 0.9280 - acc: 0.6714
For Epoch: 10
Epoch 1/10
1552/1552 [==============================] - 25s 16ms/sample - loss: 2.1634 - acc: 0.2371
Epoch 2/10
1552/1552 [==============================] - 27s 17ms/sample - loss: 1.6092 - acc: 0.4066
Epoch 3/10
1552/1552 [==============================] - 23s 15ms/sample - loss: 1.2806 - acc: 0.5329
Epoch 4/10
1552/1552 [====================

Epoch 12/20
1552/1552 [==============================] - 22s 14ms/sample - loss: 0.3667 - acc: 0.8834
Epoch 13/20
1552/1552 [==============================] - 22s 14ms/sample - loss: 0.3668 - acc: 0.8853
Epoch 14/20
1552/1552 [==============================] - 23s 15ms/sample - loss: 0.2741 - acc: 0.9182
Epoch 15/20
1552/1552 [==============================] - 25s 16ms/sample - loss: 0.2752 - acc: 0.9124
Epoch 16/20
1552/1552 [==============================] - 23s 15ms/sample - loss: 0.2483 - acc: 0.9311
Epoch 17/20
1552/1552 [==============================] - 23s 15ms/sample - loss: 0.2120 - acc: 0.9394
Epoch 18/20
1552/1552 [==============================] - 23s 15ms/sample - loss: 0.2174 - acc: 0.9336
Epoch 19/20
1552/1552 [==============================] - 22s 14ms/sample - loss: 0.1989 - acc: 0.9394
Epoch 20/20
1552/1552 [==============================] - 23s 15ms/sample - loss: 0.1621 - acc: 0.9562
For Epoch: 30
Epoch 1/30
1552/1552 [==============================] - 23s 15ms/sam

In [ ]:
tf.reset_default_graph()

In [ ]:
## Monolingual classic, Monolingual+Pre and Cross-lingual target training and evaluation
epoch_l= [1,5,10,20,30]
#ta=['gu']
lan=['sw','en','gu']   #['sw','en','gu']
lan_str=["Swahili","English","Gujarati"]   #["Swahili","English","Gujarati"]
source_tr=[X_train_sw,X_train_en,X_train_gu]  #[X_train_sw,X_train_en,X_train_gu]
target_tr=[y_train_sw,y_train_en,y_train_gu]   #[y_train_sw,y_train_en,y_train_gu]
source_te=[X_test_sw,X_test_en,X_test_gu]  #[X_train_sw,X_train_en,X_train_gu]
target_te=[y_test_sw,y_test_en,y_test_gu]   #[y_train_sw,y_train_en,y_train_gu]
for i in range(len(lan)):
    print("Target language: "+lan_str[i])
    for j in range(len(lan)):
        print("Source language: "+lan_str[j])
        for k in range(len(epoch_l)):
            print("For Epoch :"+str(epoch_l[k]))
            reset_random_seeds()
            # 1. Direct results
 #           filename='Results_Single-L3-cross-lingual/CNN-digit-mfcc-T3-E'+str(epoch_l[k])+'-S'+lan[j]+'-D'+lan[i]+'.csv'
            ##track log
#            history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)
            # Load direct models
            model_load_test = keras.models.load_model("Single-L3/CNN-digit-mfcc-T3-E"+str(epoch_l[4])+"-S"+lan[j])
            
            # Evaluate the target languages for Monolingual classic
            test_error,test_accuracy=model_load_test.evaluate(source_te[i],target_te[i],verbose=1)
            print("Accuracy on test set is: {}".format(test_accuracy))
            
            # Evaluate the target languages for Monolingual+Pre, Cross-lingual
            history=model_load_test.fit(source_tr[i],target_tr[i],validation_data=(source_te[i],target_te[i]),batch_size=32,epochs=30,verbose=1)  #,,callbacks=[history_logger]
            # 1. Direct saved model
#           model_load_test.save('Results_Single-L3-cross-lingual/CNN-digit-mfcc-T3-E'+str(epoch_l[k])+'-S'+lan[j]+'-D'+lan[i])
            
#tf.reset_default_graph()

Target language: Swahili
Source language: Swahili
For Epoch :1
388/388 [==============================] - 1s 2ms/sample - loss: 0.2808 - acc: 0.9149
Accuracy on test set is: 0.9149484634399414
Train on 1552 samples, validate on 388 samples
Epoch 1/30
1552/1552 [==============================] - 26s 17ms/sample - loss: 0.3339 - acc: 0.8834 - val_loss: 0.5493 - val_acc: 0.7964
Epoch 2/30
  64/1552 [>.............................] - ETA: 24s - loss: 0.4243 - acc: 0.8281

KeyboardInterrupt: 

In [ ]:
## Multi-lingual for direct and reverse target training and evaluation
epoch_l= [1,5,10,20,30]
#ta=['gu']
lan=['sw','en','gu']   #['sw','en','gu']
lan_str=["Swahili","English","Gujarati"]   #["Swahili","English","Gujarati"]
source_tr=[X_train_sw,X_train_en,X_train_gu]  #[X_train_sw,X_train_en,X_train_gu]
target_tr=[y_train_sw,y_train_en,y_train_gu]   #[y_train_sw,y_train_en,y_train_gu]
source_te=[X_test_sw,X_test_en,X_test_gu]  #[X_train_sw,X_train_en,X_train_gu]
target_te=[y_test_sw,y_test_en,y_test_gu]   #[y_train_sw,y_train_en,y_train_gu]
for i in range(len(lan)):
    print("Target language: "+lan_str[i])
    for j in range(len(lan)):
        print("Source language: "+lan_str[j])
        for k in range(len(epoch_l)):
            print("For Epoch :"+str(epoch_l[k]))
            reset_random_seeds()
            # 1. Direct results
 #           filename='Single-L3-multi-lingual/results/CNN-digit-mfcc-T3-E'+str(epoch_l[k])+'-S'+lan[j]+'-D'+lan[i]+'.csv'
            ##track log
#            history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)
            # Load direct models
            model_load_test = keras.models.load_model("Single-L3-multi-lingual/CNN-digit-mfcc-T3-E"+str(epoch_l[k])+"-S"+lan[j])
            
            # 2. Reverse results
#           filename='Single-L3-multi-lingual/results-reverse/CNN-digit-mfcc-T3-E'+str(epoch_l[k])+'-S'+lan[j]+'-D'+lan[i]+'.csv'
            ##track log
#            history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)
            # Load reverse models
#            model_load_test = keras.models.load_model("Single-L3-multi-lingual/reverse/CNN-digit-mfcc-T3-E"+str(epoch_l[k])+"-S"+lan[j])

            # Evaluate the target languages
            history=model_load_test.fit(source_tr[i],target_tr[i],validation_data=(source_te[i],target_te[i]),batch_size=32,epochs=30,verbose=1)  #,,callbacks=[history_logger]
            # 1. Direct saved model
#           model_load_test.save('Single-L3-multi-lingual/results/CNN-digit-mfcc-T3-E'+str(epoch_l[k])+'-S'+lan[j]+'-D'+lan[i])
            # 2. reverse saved model
#           model_load_test.save('Single-L3-multi-lingual/results-reverse/CNN-digit-mfcc-T3-E'+str(epoch_l[k])+'-S'+lan[j]+'-D'+lan[i])
            
#tf.reset_default_graph()

Target language: Swahili
Source language: Swahili
For Epoch :1
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
388/388 [==============================] - 2s 5ms/sample - loss: 2.3369 - acc: 0.0954
Accuracy on test set is: 0.09536082297563553
Train on 1552 samples, validate on 388 samples
Epoch 1/30
1552/1552 [==============================] - 26s 17ms/sample - loss: 1.9884 - acc: 0.2610 - val_loss: 2.4063 - val_acc: 0.0954
Epoch 2/30
1552/1552 [==============================] - 24s 16ms/sample - loss: 1.4568 - acc: 0.4659 - val_loss: 2.4440 - val_acc: 0.0954
Epoch 3/30
1552/1552 [==============================] - 26s 16ms/sample - loss: 1.1137 - acc: 0.5960 - val_loss: 2.3499 - val_acc: 0.1005
Epoch 4/30
1552/1552 [==============================] - 26s 17ms/sample - loss: 0.9088 - acc: 0.6765 - val_loss: 2.2768 - val_acc: 0.1005
Epoch 5/30
1552/1552

KeyboardInterrupt: 

In [ ]:
# Proposed method
epoch_l=[1,5,10,20,30]
n=[4,7]  # Conv block sequence types A1-B2-B3 (4) and A1-A2-B3 (7) 

# Target languages
s=['sw','en','gu'] 
# Source languages
lan=['gu','sw']   #Swahili: ['en','gu']   English: ['gu','sw']  Gujarati: ['sw','en']
# Select Target language index no 0,1,2 : Swahili, English, Gujarati
indno=1
# Language strings
lan_str=["Swahili","English","Gujarati"]
# Training set
source_tr=[X_train_sw,X_train_en,X_train_gu]  
target_tr=[y_train_sw,y_train_en,y_train_gu]   
# Test set
source_te=[X_test_sw,X_test_en,X_test_gu]  
target_te=[y_test_sw,y_test_en,y_test_gu]   


for i in range(len(lan)):
    print("Early layers language: "+lan_str[i])
    for j in range(len(epoch_l)):
        print("Epoch :"+str(epoch_l[j]))
        for k in range(len(n)):
            print("Number of layers: "+str(n[k]))
            reset_random_seeds()
            
            # Upload source 1 Pre-training weights = source data features
            model_load_test = keras.models.load_model("Single-L3/CNN-digit-mfcc-T3-E"+str(epoch_l[0])+"-S"+lan[i])
            # Upload source 2 Pre-training weights = target data features
            # Alternate epochs source numbers [1,5,10,20,30]
            model_load_test_f = keras.models.load_model("Single-L3/CNN-digit-mfcc-T3-E"+str(epoch_l[4])+"-S"+s[indno])  #CNN-digit-mfcc-g-sw_gu-d-16-
            
            # Mixing the weights of source 1 and source 2
            # Select Conv block sequence types A1-B2-B3 and A1-A2-B3 
            for l1,l2 in zip(model_load_test_f.layers[0:n[k]],model_load_test.layers[0:n[k]]):  
                l1.set_weights(l2.get_weights())   # Mixing the weights
      
 #           filename='Results_Single-Mix-gu/1/CNN-digit-mfcc-T3-E'+str(epoch_l[j])+'-S'+lan[i]+'gu-'+str(n[k])+'-Dgu.csv'
            #track log
#           history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)
            # The Target Gujarati training and Evaluating. 
            history=model_load_test_f.fit(source_tr[indno],target_tr[indno],validation_data=(source_te[indno],target_te[indno]),batch_size=32,epochs=30,verbose=1) #,callbacks=[history_logger]
            

Early layers language: Swahili
Epoch :1
Number of layers: 4
Train on 1552 samples, validate on 388 samples
Epoch 1/30
1552/1552 [==============================] - 26s 17ms/sample - loss: 0.2234 - acc: 0.9265 - val_loss: 6.5440 - val_acc: 0.1263
Epoch 2/30
1552/1552 [==============================] - 30s 19ms/sample - loss: 0.1463 - acc: 0.9510 - val_loss: 6.5873 - val_acc: 0.1753
Epoch 3/30
1552/1552 [==============================] - 40s 26ms/sample - loss: 0.1012 - acc: 0.9671 - val_loss: 6.5354 - val_acc: 0.1160
Epoch 4/30
1552/1552 [==============================] - 28s 18ms/sample - loss: 0.0608 - acc: 0.9807 - val_loss: 4.9575 - val_acc: 0.1366
Epoch 5/30
1552/1552 [==============================] - 31s 20ms/sample - loss: 0.0514 - acc: 0.9865 - val_loss: 5.0245 - val_acc: 0.1624
Epoch 6/30
1552/1552 [==============================] - 33s 21ms/sample - loss: 0.0498 - acc: 0.9878 - val_loss: 4.2244 - val_acc: 0.2216
Epoch 7/30
1552/1552 [==============================] - 28s 18ms/

KeyboardInterrupt: 